In [ ]:
# ignore warnings

import warnings
warnings.filterwarnings("ignore")

Installing Dependencies

In [ ]:
! pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
! pip install datasets
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 38.6 MB/s eta 0:00:01tcmalloc: large alloc 1982210048 bytes == 0x241a000 @  0x7f4e8dc381e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 32.5 MB/s eta 0:00:01tcmalloc: large alloc 2477768704 bytes == 0x7867c000 @  0x7f4e8dc39615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91

Load dataset

In [ ]:
from datasets import load_dataset
multi_news = load_dataset("multi_news", split="test")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


View Data

In [ ]:
multi_news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


Train test split (80:20 ratio respectively)

In [ ]:
multi_news = multi_news.train_test_split(test_size=0.2)

Tokenizing Training and testing sets

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_multi_news = multi_news.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Model Loading

In [ ]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Defining Hyperparameters

In [ ]:
training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=10,
per_device_eval_batch_size=10,
weight_decay=0.01,
save_total_limit=3,
num_train_epochs=10,
fp16=True,
)


Initializing Trainer

In [ ]:
trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized_multi_news["train"],
eval_dataset=tokenized_multi_news["test"],
tokenizer=tokenizer,
data_collator=data_collator,
)


Using cuda_amp half precision backend


Train the Text Summarization Model

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.871367
2,3.259500,2.807521
3,2.994900,2.781594
4,2.943500,2.763112
5,2.919300,2.751168
6,2.908200,2.744571
7,2.888100,2.741190
8,2.862300,2.737068
9,2.875300,2.735423
10,2.864300,2.735541


TrainOutput(global_step=4500, training_loss=2.946165690104167, metrics={'train_runtime': 3278.8303, 'train_samples_per_second': 1.372, 'total_flos': 0, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 3226583040, 'init_mem_gpu_alloc_delta': 242026496, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 69242880, 'train_mem_gpu_alloc_delta': 735640064, 'train_mem_cpu_peaked_delta': 26296320, 'train_mem_gpu_peaked_delta': 9485109760})

Evaluate model on a single document

In [ ]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [ ]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [ ]:
human_summary

'– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today\'s jobs report. Reaction on the Wall Street Journal\'s MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment.'

In [ ]:
predicted_summary = predict_summary(document)
predicted_summary

'<pad><extra_id_0>, and the New York Times reports that the economy has added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2%, the lowest since January 2009. The Federal Reserve Chairman has cautioned that the current hiring pace is unlikely to continue without more consumer spending. The New York Times reports that the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2%, the lowest since January 2009. The unemployment rate is expected to hold steady at 8.3%. The New York Times reports'

Evaluate using Rouge Scores

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a suite of metrics for evaluating the quality of text summaries. It compares a set of reference summaries with a set of generated summaries, and computes a score based on the overlap between the two.

In [ ]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [ ]:
# import validation dataset

import pandas as pd
val_data = pd.read_excel('/content/Validation_Data.xlsx')
val_data

,document,summary
0,"Story highlights Officers circled the man, who...","– Angry relatives are calling it an ""execution..."
1,Ebola patient in critical condition in Texas h...,– The Ebola patient who's in critical conditio...
2,Skip in Skip x Embed x Share CLOSE President O...,"– Hold your fire, troops: President Obama made..."
3,William Aitcheson stepped down after revealing...,– A Catholic priest in Virginia has stepped do...
4,Taylor Swift's epic treadmill fall in Apple Mu...,"– On April 1, Taylor Swift shared an ad for Ap..."
...,...,...
95,Locked-In Man \n \n Invisibilia's Lulu Miller ...,– Martin Pistorius was a kid so into electroni...
96,Iraqi security forces celebrate as they pose w...,– Iraqi special forces swept into Fallujah on ...
97,Treasury Secretary Timothy F. Geithner said on...,– Tim Geithner is about to enact some “extraor...
98,"Votes for Mickey Mouse, football coach Nick Sa...",– On Wednesday Vice President Mike Pence will ...


In [ ]:
# rouge score of validation data

from tqdm import tqdm

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(val_data))):

  doc = val_data.loc[i]['document']
  pred_summary = predict_summary(document)
  human_summary = val_data.loc[i]['summary']

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

val_data["pred_summary"] = pred_summary_list

val_data['rouge1'] = rouge1_scores
val_data['rouge2'] = rouge2_scores
val_data['rougel'] = rougel_scores

val_data

100%|██████████| 100/100 [01:59<00:00,  1.20s/it]


,document,summary,rouge1,rouge2,rougel,pred_summary
0,"Story highlights Officers circled the man, who...","– Angry relatives are calling it an ""execution...",0.104265,0.013289,0.094787,"<pad><extra_id_0>, and the New York Times repo..."
1,Ebola patient in critical condition in Texas h...,– The Ebola patient who's in critical conditio...,0.117117,0.006309,0.117117,"<pad><extra_id_0>, and the New York Times repo..."
2,Skip in Skip x Embed x Share CLOSE President O...,"– Hold your fire, troops: President Obama made...",0.086486,0.000000,0.086486,"<pad><extra_id_0>, and the New York Times repo..."
3,William Aitcheson stepped down after revealing...,– A Catholic priest in Virginia has stepped do...,0.119266,0.020270,0.110092,"<pad><extra_id_0>, and the New York Times repo..."
4,Taylor Swift's epic treadmill fall in Apple Mu...,"– On April 1, Taylor Swift shared an ad for Ap...",0.149425,0.018265,0.126437,"<pad><extra_id_0>, and the New York Times repo..."
...,...,...,...,...,...,...
95,Locked-In Man \n \n Invisibilia's Lulu Miller ...,– Martin Pistorius was a kid so into electroni...,0.072131,0.004505,0.072131,"<pad><extra_id_0>, and the New York Times repo..."
96,Iraqi security forces celebrate as they pose w...,– Iraqi special forces swept into Fallujah on ...,0.096491,0.006452,0.096491,"<pad><extra_id_0>, and the New York Times repo..."
97,Treasury Secretary Timothy F. Geithner said on...,– Tim Geithner is about to enact some “extraor...,0.106509,0.000000,0.106509,"<pad><extra_id_0>, and the New York Times repo..."
98,"Votes for Mickey Mouse, football coach Nick Sa...",– On Wednesday Vice President Mike Pence will ...,0.106557,0.012579,0.090164,"<pad><extra_id_0>, and the New York Times repo..."


In [ ]:
# average rouge 1
val_data['rouge1'].mean()

0.11723885568649839

In [ ]:
# average rouge 2
val_data['rouge2'].mean()

0.012346104257379096

In [ ]:
# average rouge l
val_data['rougel'].mean()

0.1114219289972185